# XGBoost



## Learning Objective
- Explain XGboost as a fast and scalable machine learning library.
- Understand problems with typical decision tree
- Understand problems with typical boosting algorithms
- Explain how XGBoost fixes the issues with a typical decision tree and boosting algorithms.
- List out different features of XGBoost.


## Introduction

XGBoost stands for __Extreme Gradient Boosting__. It is a gradient boosting based library designed for speed and performance. Chen T. et al., in 2016, published a paper `XGBoost: A Scalable Tree Boosting System` where they have introduced XGBoost as a scalable machine learning system for tree boosting. The author states that the reason for the popularity of this model is its scalability.


You might be familiar with popular machine learning competitions like KDD cup, and Kaggle competition. XGBoost is popular for winning these competitions most of the time. It can be used for regression, classification, and ranking tasks.


In the upcoming sections, we will briefly discuss the problems/bottlenecks of decision trees and general boosting algorithms. Then we will talk about how XGBoost addresses the issues with decision tree and general boosting.

## Problems with decision tree

XGBoost is a very good package, and it fixes several bottlenecks of decision tree and general boosting algorithm. When dealing with large scale machine learning problem, we start realizing the bottleneck of a decision tree and a general boosting algorithm.  Let's first discuss a few problems with the decision tree and how XGBoost address them. Later we will discuss the problems with general boosting.





        

### 1. Greedy Split Finding

The first problem with the decision tree is its greedy split finding process. We have studied how the decision tree creates split in the Decision Tree module. At each node, the best split is determined by iterating through all the values of all the features in the decision tree.
This process is called greedy split finding. Greedy split finding technique is a bottleneck in term of computational requirement when dealing with a large scale problem containing millions of rows and hundreds of features.

Instead of using an exact greedy approach, XGBoost uses __approximate algorithm__. In this method, candidate split points are selected according to the quantiles of feature distribution or simply based on data statistics. For instance, we compute the first, second, and third quartile of some continuous features, propose them as candidate split points, and select the best split point among those proposed candidate points. This mechanism drastically reduces the number of candidate splits.

For example, a feature has 100 unique values with values $\{1,2,3,...,100\}$. In decision tree, we iterate through all these unique values to find the best value. That means we first sort these values, iterate through each of these values,  take each value as a threshold, and check if that particular selection of threshold creates a better split. However, in XGBoost, the approximation method proposes a candidate split points based on the quantile. For instance, with 10 quantiles, the candidate split points are $\{10,20,30,...,90\}$. Now we check for split at these 9 points only. This mechanism drastically reduces the computation while the performance remains almost still the same. If we increase the number of quantiles, say $\{5, 10, 15,...., 95\}$, then we can do a more accurate split.


Similarly, if we have tons of data such that it is difficult to fit them in a computer at one time, then process like sorting the data and finding the quantile is slow. To solve this issue, a method called quantile sketch is used. In this method, huge data is divided into smaller chunks and are placed on different computers. The quantile sketch algorithm then combines the data statistics from each computer to create an approximate histogram. This histogram is used to calculate the approximate quantiles. This is a normal quantile sketch method.

<figure>
    <div align = "center">
 <!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=1B7BPEp7_rb2xem7YzpsiO9aELD8zDes6" width="%"> -->
  <img src="https://i.postimg.cc/jjcCF001/image.png" width="%">
    </div>
    <div align = "center">
<figcaption>Figure 1: Work flow of quantile sketch algorithm <figcaption>
    </div>
 </figure>

However, XGBoost uses a __weighted quantile sketch method__, which is slightly different from the normal quantile sketch method. The number of samples in different quantile are equal in the normal quantile sketch method. But in weighted quantile sketch, number of samples in different quantiles can be different.





### 2. Overfitting

Very deep decision trees are highly prone to overfitting.  XGBoost introduces an explicit regularization term to reduce overfitting term. Apart from the regularization term, XGBoost introduces other techniques like column sub-sampling as in the random forest, row-subsampling as in bagging to reduce overfitting. We will discuss different techniques introduced in XGBoost to avoid overfitting in the coming section.



### 3. Decision Tree Strucutre:

The typical decision tree structure is not friendly for performing boosting. For example, the decision tree for classification only returns the output classes. For boosting, the output of the tree should be some real value. For solving this issue, XGBoost provides a new representation of a tree called the regression tree, which gives the real value at each leaf, also called weight($w$).


Suppose we use a decision tree to predict if someone likes a particular computer game based on different features like age, gender, occupation, etc.

<figure>
    <div align = "center">
 <img src="https://i.postimg.cc/8C7PtF47/image.png" width="%">
    </div>
    <div align = "center">
<figcaption>Figure 2: Tree representation with weights (Source: <a href='https://arxiv.org/pdf/1603.02754.pdf'>XGBoost: A Scalable Tree Boosting System</a>)  <figcaption>
    </div>
 </figure>
Suppose we create a shallow decision tree with each instance assigned to different leaves, as shown in the figure. Note that each leaf is assigned a real prediction score called weight. The real score gives a much richer representation(or interpretation) that goes beyond a typical classification. For instance, representing each leaf by the majority class doesn't tell how well the prediction of a particular node is. However, with real value, we can quantify each leaf. Moreover, representing each leaf by the real score provides us with a unified approach to solving both classification and regression problems. That means we can solve the classification problem in the same way we solve regression. Please refer the paper on XGBoost in the Additional Resources section at the end of the reading material to know how to compute the weight.

See [this](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf) for more details on regression tree.
        

## Problem with boosting
In the previous chapters, we have discussed boosting and different algorithms like AdaBoost and gradient boosting. We have talked about boosting as a powerful algorithm, but they have some problems with them. One of the major problems with boosting is its sequential nature. Let's discuss it.

### 1. Sequential Nature

Unlike bagging, boosting is sequential by nature. Each base learner is subsequently added based on the output of an existing model. This makes boosting slow. To accelerate, XGBoost uses different techniques like parallelization and distributed computing. XGBoost introduces parallelism while constructing each tree. But multiple trees can't be built parallelly. XGBoost uses all cores to build the first tree; once the first tree is built, it uses all the cores to build the second tree and so on, as shown in the figure below.

<figure>
    <div align = "center">
 <!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=15fpKEM-7rCMUBM4nhdxz2tk098BWmE9Z" width="50%"> -->
 <img src="https://i.postimg.cc/FFp5krb8/image.png" width ="50%">
    </div>
    <div align = "center">
<figcaption>Figure 3: Figure showing the use of all cores while creating each tree<figcaption>
    </div>
 </figure>

 XGBoost uses a block structure that enables parallelization while building each tree. We will discuss this in the System Features section.    

## Features or improvement in XGBoost

XGBoost introduces different features or improvements. Some features are introduced to address the problem with the decision tree and general boosting framework. Similarly, other features are added to make XGBoost more useful and powerful. In the above sections, we discussed the shortcomings with both the decision tree and the general boosting framework. Let's discuss different features XGBoost has introduced to address problems with decision trees and typical boosting.

## 1. Regularization

A deep decision tree is highly prone to overfitting. To deal with this problem, XGBoost introduces an explicit regularization term.

 In gradient boosting, we optimize a loss function using a gradient descent algorithm. The expression for objective function in gradient boosting is:
$$ Obj^t = \sum_{i=1}^nL(y_i,F_t(\mathbf{x}_i))$$
Where, superscript $t$ is used to denote $t^{th}$ boosting iteration.

 __XGBoost__ adds a regularization parameter in the objective function which is given by:

 $$Obj^t = \sum_{i=1}^nL(y_i,F_t(\mathbf{x}_i))+ \Omega(h_t)$$

 The second term $\Omega$ in the above equation is a regularization function, which measures the model's complexity. It allows us to control the complexity of our model.  Two variants of regularization are popular; they are $L1$ norm and $L2$ norm.

$\text{L1  norm : } \Omega(v)= \lambda\space||v||$

$\text{L2  norm : } \Omega(v)= \lambda\space||v||^2$


With L2 norm, the regularization function is:

$$\Omega(h) = \gamma T+\frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$$
$$\text{Here, }T \text{ is the number of leaves}$$

Where, $w_j$ represents the weight associated with each leaf($j$) in the tree.

The first expression on the right-hand side of the regularization function limits the growth of the tree. That means it puts extra punishment for unnecessarily large or complex decision trees. Similarly, the second expression on the right-hand side of the regularization function limits each leaf's prediction score or weight in a tree.

## 2. Second Order gradient

For optimization purposes, the classical gradient boosting uses only the first-order derivative term to decide on how to add successive models. However, XGboost considers both the first and second-order derivative while deciding on how to add successive models.
It means, in XGBoost each successive tree $h$ is trained on $\big({\mathbf{x}_i, -\frac{g_i}{h_i+\lambda}}\big)_{i=1}^N$.

where,

$$
g_i = \partial_{F_{t-1}(\mathbf{x}_i)}L\big(y_i, F_{t-1}(\mathbf{x}_i)\big) \quad \text{is the first order derivative, called gradient}
$$

and

$$
h_i = \partial^2_{F_{t-1}(\mathbf{x}_i)}L\big(y_i, F_{t-1}(\mathbf{x}_i)\big) \quad \text{is the second order derivative, called Hessian}
$$

This consideration of the second-order derivative term and the first-order derivative term improve convergence speed towards the minima.

## 3. Sparsity Aware Split Finding

Most of the real-world data suffer from sparsity. This is mainly due to the presence of missing values and frequent zero-entry during data collection. We need to fill those missing values with other machine learning algorithms, but this is not the case with XGBoost. XGBoost has a provision of default direction for the missing value. It learns the default direction/path for the missing values during training.

<figure>
    <div align = "center">
 <!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=1Jdfu8Vx1SSVYOyA4P8d4eQKkjKp7FZzL" width="%"> -->
 <img src="https://i.postimg.cc/KYTRrqmK/image.png" />
    </div>
    <div align = "center">
<figcaption>Figure 4: Tree showing default direction for missing values<figcaption>
    </div>
 </figure>



Let's discuss how XGBoost handles this.
First, trees are built with the help of instances with no missing values. Now we gather all the instances with a missing value. For each node, the missing instances are first directed to the left child node, and we calculate the gain. Similarly, these instances are now directed to the right child node, and the value of gain is calculated. The left or the right direction, whichever obtains the better gain, is set as a default direction for that particular attribute with a missing value.

## 4. Stochastic Gradient Boosting

Stochastic Gradient Boosting is an improvement over classical gradient boosting used in XGBoost. This improvement helps to reduce overfitting. Stochastic property is added in XGBoost using two techniques: Row subsampling and column subsampling.

### Row subsampling

Like Bagging, XGBoost subsamples a certain portion of the training dataset to train each tree. This reduces the correlation among individual trees by a certain ratio. For example, if we set row subsampling to 0.6, 60% of the training samples are randomly sampled to create each tree.

### Column subsampling

Like a random forest, XGBoost has a provision of column subsampling where features are randomly sampled, and the best split point is searched among those sampled features. XGBoost provides parameters like __colsample_bytree__, __colsample_bylevel__, and __col_sample_bynodes__ for different level of column sampling.

__colsample_bytree__ controls the number of features used in each tree. Features are sampled while building each tree.

__colsample_bylevel__ controls the number of features used in each level(depth). Subsampling occurs from features already subsampled in __colsample_bytree__ for each new depth reached.


Similarly, __colsample_bynode__ controls the number of features sampled from features already sampled in __colsample_bylevel__ when finding the best split at each node. Features for a node are sampled from the list of features already sampled for that level.

These parameters act cumulatively. For example,

Suppose we have 16 features in total and we set parameters as
 __colsample_bytree__=0.5, __colsample_bylevel__=0.5, __colsample_bynode__=0.5, then,

 Number of features sampled for each tree = __colsample_bytree__ * 16=8

 Number of features sampled for each level in a tree = __colsample_bytree__* __colsample_bylevel__ * 16 = 4

 Number of features sampled for each node in each level = __colsample_bytree__* __colsample_bylevel__ * __colsample_bynode__ * 16 = 2

### $\textbf{5. Shrinkage:}$

The contribution of each subsequently added tree is damped or shrunk by applying shrinkage or learning_rate. The shrinkage helps to avoid overfitting.  
  
In the optimization function discussed above, we didn't introduce the shrinkage parameter for simplicity, but it is present in the objective function. The actual objective function, including the shrinkage factor, is:

$$
Obj^t = \sum_{i=1}^n L\left(y_i, F_{t-1}(\mathbf{x}_i) + \epsilon h_t(\mathbf{x}_i)\right) + \Omega(h_t)
$$


  Where $\epsilon$ is step-size or shrinkage or learning_rate.



### 6. Custom Loss Function

XGBoost allows us to define our custom loss function. Various loss functions like absolute error, squared loss, logistic loss, etc. are already available. We can specify which loss function to use for optimization. For regression, the default loss function is squared error.

$\textbf{For regression:}$

$\text{Squared loss : } L(y_i, F(\mathbf{x}_i)) = \big(y_i - F(\mathbf{x}_i)\big)^2$


$\textbf{For binary classification:}$

 $\text{Logistic loss : } L(y_i,F(\mathbf{x}_i)) = y_ilog(1+e^{-F(\mathbf{x}_i)})+ (1-y_i)log(1+e^{F(\mathbf{x}_i)})$

## 7. System Features

Even though XGBoost is a boosting based library, it is quite fast. It is so because of various system optimization. Let's discuss different system features:

### Parallelization

XGBoost uses an in-memory unit called blocks to store data. A block of each feature is formed where feature value are stored in sorted order(ascending order). In block structure, blocks are sorted once at the start of training and can be reused at successive iteration. For finding the best split, we scan through these sorted data independently in each block structure. This allows us to apply parallelization (or multithreading) while scanning through each block for finding the best split.


### Distributed Computing

XGBoost has provisions for using multiple machines in a distributed environment while dealing with large scale dataset.

### Cache Optimization

XGBoost allows cache-aware prefetching of gradient and hessian statistics in its internal buffer.

### Out-of-Core-Computation

When the problem at hand is large, the training data might not fit in the RAM. XGBoost has a provision of fetching data directly from the disk. However, fetching the data directly from disk is a bottleneck in terms of time complexity. XGBoost uses different techniques like block compression, block sharding, etc. to improve these time constraints. The discussion of these techniques is out of scope in this notebook.

### Key Takeaways

- XGBoost is an implementation of gradient boosting for performance and scalability.

- As an alternative to the greedy split finding method, XGBoost uses an approximation algorithm.

- XGBoost uses regularization, shrinkage, row sub-sampling, and column-subsampling to reduce the overfitting.

- The use of a second-order derivative term for optimization helps in faster convergence.

- System features like parallelization, distributed computing, cache optimization, and out-of-core computation makes XGBoost fast, powerful, and scalable.
